In [1]:
import numpy as np
import pandas as pd
from scipy.spatial.transform import Rotation as R

from scipy import optimize
from sklearn.neighbors import KDTree
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.neighbors import BallTree
from tqdm.notebook import tqdm

from readers_writers import *

# Create not optimized membrana

### all functions you need for creating not optimized mem

In [6]:
np.random.seed(41)

def angle(vector_1, vector_2): # find angle between vectors, radian
    unit_vector_1 = vector_1 / np.linalg.norm(vector_1)
    unit_vector_2 = vector_2 / np.linalg.norm(vector_2)
    dot_product = np.dot(unit_vector_1, unit_vector_2)
    return np.arccos(dot_product)

def rotate_system(a: np.array, b: np.array, coord: np.array) -> np.array:
    """
    Parameters
    ----------
    a : rotatable vector
    b : direction vector
    coord : coordinations of our system
    Returns
    -------
    System will be rotate at the same angle as we rotate vector a to make it parallel with vector b
    """
    if np.linalg.norm(np.cross(a, b)) == 0: # calc mult vector, in case of parallel vector we choose any normal vector
        if a[0] == 0:
            rv = [np.pi, 0, 0]
        else:
            rv = np.array([-(a[1]+a[2])/a[0], 1, 1])/np.linalg.norm([-(a[1]+a[2])/a[0], 1, 1]) * np.pi
    else:
        rv = np.cross(a, b)/np.linalg.norm(np.cross(a, b)) * angle(a, b)
    
    r = R.from_rotvec(rv)
    return r.apply(coord)


# |------------------------- build membrana -------------------------|
def add_mol(df, system, r_c, i=0, j=0):
    new_mol = df.copy()
    random_angle = np.random.random()*360
    r = R.from_euler('x', random_angle, degrees=True)
    new_mol[['x', 'y', 'z']] = r.apply(new_mol[['x', 'y', 'z']])
    new_mol[['x', 'y', 'z']] += np.array([0, r_c, r_c]) * np.array([0, i, j])
    new_mol['resnum'] = system.resnum.iloc[-1] + 1
    return pd.concat([system, new_mol])


def make_layer(molecules, system, r_c=14.5, d_y=7, d_z=7):
    n = 1
    for i in tqdm(range(d_y)):
        for j in range(d_z):
            if i+j == 0:
                continue
            system = add_mol(molecules[i][j], system, r_c, i=i, j=j)
            n += 1
    
    system['atnum'] = np.arange(1, system.shape[0] + 1)
    return system


def gen_layers_pattern(k=6, eu=70, w=30):
    pattern_layer0 = [0 for i in range(eu)] + [1 for i in range(w)]
    np.random.shuffle(pattern_layer0)
    pattern_layer0 = np.array(pattern_layer0)
    pattern_layer1 = np.where(pattern_layer0 == 1, 0, 1)
    idx = np.argwhere(pattern_layer1 == 1)
    np.random.shuffle(idx)
    idx = idx[:eu - w]
    pattern_layer1[idx] = 0
    pattern_layer0, pattern_layer1 = pattern_layer0.reshape((k, k)), pattern_layer1.reshape((k, k))
    pattern_layer0 = pattern_layer0[:, ::-1]
    return pattern_layer0, pattern_layer1

# |-----------for optimize molecule location -------------|

def e(df):
    return (((df.z**2 + df.y**2))**0.5).max()

def rotate_system_foropt(angles, df):
    r = R.from_euler('xyz', angles, degrees=True)
    df[['x', 'y', 'z']] = r.apply(df[['x', 'y', 'z']])
    return df

def f(angles, df):
    dft = df.copy()
    dft = rotate_system_foropt(angles, dft)
    return e(dft)

def rotate_system_opt(df):
    lw = [-90] * 3
    up = [90] * 3
    bounds=list(zip(lw, up))
    res = optimize.dual_annealing(f, bounds=bounds, maxiter=100, args=(df,))
    
    df = rotate_system_foropt(res.x, df)
    print(e(df))
    return df

In [4]:
# |------------------------ lets ROCK! ------------------ |
# read molecules and rotate them as it is needed by guiding atoms

df = read_pdb('files/KMA_W.pdb', ) 
atoms = (df[df.atom == 'O66'].index[0],
         df[df.atom == 'C59'].index[0])

df.loc[:, ['x', 'y', 'z']] = df.loc[:, ['x', 'y', 'z']] - df.loc[[atoms[0]], ['x', 'y', 'z']].values
df[['x', 'y', 'z']] = rotate_system(df.loc[[atoms[1]], ['x', 'y', 'z']], np.array([1, 0, 0.]), df[['x', 'y', 'z']])
df[['x', 'y', 'z']] -= df[['x', 'y', 'z']].mean()

df1 = read_pdb('files/AMA_eU.pdb')
atoms = (df1[df1.atom == 'O81'].index[0],
         df1[df1.atom == 'C70'].index[0]) 

df1.loc[:, ['x', 'y', 'z']] = df1.loc[:, ['x', 'y', 'z']] - df1.loc[[atoms[0]], ['x', 'y', 'z']].values
df1[['x', 'y', 'z']] = rotate_system(df1.loc[[atoms[1]], ['x', 'y', 'z']], np.array([1, 0, 0.]), df1[['x', 'y', 'z']])
df1[['x', 'y', 'z']] -= df1[['x', 'y', 'z']].mean()

df2 = read_pdb('files/MMA_good_W.pdb')
atoms = (df2[df2.atom == 'O65'].index[0],
         df2[df2.atom == 'C18'].index[0])

df2.loc[:, ['x', 'y', 'z']] = df2.loc[:, ['x', 'y', 'z']] - df2.loc[[atoms[0]], ['x', 'y', 'z']].values
df2[['x', 'y', 'z']] = rotate_system(df2.loc[[atoms[1]], ['x', 'y', 'z']], np.array([1, 0, 0.]), df2[['x', 'y', 'z']])
df2[['x', 'y', 'z']] -= df2[['x', 'y', 'z']].mean()



# choose the best rotation
df = rotate_system_opt(df)
df1 = rotate_system_opt(df1)
df2 = rotate_system_opt(df2)


9.613614458000955
5.455396531150759
7.160477797638107


In [7]:
# In some membranes, we cannot place 
# some molecules on top of others.

mem_size = 10
pattern_layer0, pattern_layer1 = gen_layers_pattern(mem_size, 50, 50)

In [8]:
pattern_layer0 = [list(i) for i in pattern_layer0]
pattern_layer1 = [list(i) for i in pattern_layer1]

for arr in pattern_layer0, pattern_layer1:
    eu = [df1.copy() for _ in range(50)] 
    w = [df.copy() for _ in range(25)] + [df2.copy() for _ in range(25)]
    np.random.shuffle(w)
    for i in range(len(arr)):
        for j in range(len(arr[0])):
            if arr[i][j] == 0:
                arr[i][j] = eu.pop()
            else:
                arr[i][j] = w.pop()

# Create box and show parametres

In [9]:
layer0 = make_layer(pattern_layer0, pattern_layer0[0][0], 14, mem_size, mem_size)
layer1 = make_layer(pattern_layer1, pattern_layer1[0][0], 14, mem_size, mem_size)

r = R.from_euler('y', 180, degrees=True)
mean = layer1[['x', 'y', 'z']].mean()
layer1.loc[:, ['x', 'y', 'z']] -= mean
layer1.loc[:, ['x', 'y', 'z']] = r.apply(layer1[['x', 'y', 'z']])
layer1.loc[:, ['x', 'y', 'z']] += mean
layer1['x'] += layer0.x.max() - layer1.x.min()


layer1['resnum'] += layer0.resnum.max()
system = pd.concat([layer0, layer1])

# desc = system.loc[:, ['x', 'y', 'z']].describe()
# a = desc.loc['max'] - desc.loc['min']
# v = a['x'] * a['y'] * a['z']
# masses = {'H': 1, 'C': 12, 'N': 14, 'O': 16}
# system['m'] = system.atType.apply(lambda x: masses[x])
# print('p =', system.m.sum()/v * 1660)
# d = system[['x', 'y', 'z']].describe()
# print('box size is\n', d.loc['max'] - d.loc['min'], sep='')
# mass = []
# system.to_csv('not_opt.csv', index=False)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [10]:
write_pdb(system)

# Optimization of membana
I use dual anneling, lennard - johnes potential as optimizer 
parametres - angles and movments of molecule

In [6]:
%%time
def write_pdb(system, file='test.pdb'):
    system['atType'] = system.atType.apply(lambda x: x[0])
    with open(file, 'w') as file:
        for i in range(system.shape[0]):
            if i > 1 and system.iloc[i].resnum != system.iloc[i-1].resnum:
                file.write('TER\n')

            file.write('HETATM{:5d} {:>4s} {:<4s} {:<4d}    {:8.3f}{:8.3f}{:8.3f}  0.00  0.00               {:s}\n'
                       .format(*system.iloc[i][['atnum', 'atom', 'res', 'resnum', 'x', 'y', 'z', 'atType']]))
        file.write('END\n')
        

def choose_res(df, resnum):
    res = df.loc[df.resnum == resnum, ['x', 'y', 'z']].values
    system = df.loc[df.resnum != resnum, ['x', 'y', 'z']].values
    return system, res


def make_tree(system):
    return BallTree(system, leaf_size=3)


def calc_dist(tree, X):
    dist, _ = tree.query(X, k=10)
    dist = np.where(dist < 4, dist, 4)
    return dist

d = 3.5
sig = d / 2**(1/6)
e = 1e1
pot612 = lambda r: 4*e*( (sig/r)**12 - (sig/r)**6 )

def E(dist):
    return pot612(dist).sum()


def move_rot(X, res):
    res = res.copy()
    r = R.from_euler('x', X[0], degrees=True)
    res_mean = res.mean(axis=0)
    res -= res_mean
    res = r.apply(res)
    res += res_mean
    res += X[1:]
    return res


def f(X, res, tree):
    new_res = move_rot(X, res)
    return E(calc_dist(tree, new_res))



def opt(df, res_to_opt, nit=1000):
    system, res = choose_res(df, res_to_opt)
    tree = make_tree(system)
    lw = [0, -1.5, -1.5, -0.1]
    up = [360, 1.5, 1.5, 0.1]
    bounds=list(zip(lw, up))
    result = optimize.dual_annealing(f, bounds=bounds, maxiter=nit, args=(res, tree))
    new_res = move_rot(result.x, res)
    new_df = df.copy()
    new_df.loc[new_df.resnum == res_to_opt, ['x', 'y', 'z']] = new_res
    i = res_to_opt
    dft0 = df.loc[df.resnum == i, ['x', 'y', 'z']]
    dft1 = df.loc[df.resnum != i, ['x', 'y', 'z']]
    d_old = euclidean_distances(dft0, dft1).min()
    dft0 = new_df.loc[new_df.resnum == i, ['x', 'y', 'z']]
    dft1 = new_df.loc[new_df.resnum != i, ['x', 'y', 'z']]
    d_new = euclidean_distances(dft0, dft1).min()
    if d_old < d_new:
        return new_df
    # print('FAIL')
    return df


df = pd.read_csv('not_opt.csv')

gg = df.groupby('resnum').agg(x=('x', 'mean'), y=('y', 'mean'), z=('z', 'mean')).reset_index()
print(len(set(gg.query('y > 52 or y < -52 or z > 52 or z < -52').resnum)))
for j in range(2):
    for i in tqdm(range(1, 201)):
        if i not in set(gg.query('y > 52 or y < -52 or z > 52 or z < -52').resnum):
            df = opt(df, i, 100)
    df.to_csv(f'files/w_{j}.csv')

72


  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

Wall time: 16min 34s


### Finally we have got this:

<img src="membrana.png">